## 문제 4-1 : OpenAI에서 Ollama Qwen3로 RAG 시스템 변경하기

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings # 변경: HuggingFaceEmbeddings 임포트
from langchain_ollama import ChatOllama # 변경: ChatOllama 임포트
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

print("모든 라이브러리 임포트 완료.")

모든 라이브러리 임포트 완료.


In [3]:
print("==> 1. 문서 로딩 → PDF 읽기...")
loader = PyPDFLoader('../data/tutorial-korean.pdf')
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")

==> 1. 문서 로딩 → PDF 읽기...
  총 39페이지 로드 완료


In [4]:
print("==> 2. 문서 분할 → 작은 청크로 나누기")
# 로컬 모델을 위한 최적화
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # 변경: 1000 -> 800 (조금 더 작게)
    chunk_overlap=150, # 변경: 200 -> 150 (중복 감소)
    separators=["\n\n", "\n", ".", " ", ""]
)

chunks = text_splitter.split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")

==> 2. 문서 분할 → 작은 청크로 나누기
  76개 청크 생성 완료


In [5]:
print("==> 3. 벡터화 → 임베딩으로 변환")
# 임베딩 모델 변경: OpenAIEmbeddings -> HuggingFaceEmbeddings
EMBEDDING_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs={'device': 'cpu'}, # CPU 사용 명시 (GPU가 있다면 'cuda'로 변경 가능)
    encode_kwargs={'normalize_embeddings': True}
)
print(f"  '{EMBEDDING_MODEL_NAME}' 임베딩 모델 로드 완료.")

==> 3. 벡터화 → 임베딩으로 변환


c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\langchain-basic-kGdHTiMZ-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2' 임베딩 모델 로드 완료.


In [6]:
print("==> 4. 저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, embeddings)
print("  FAISS 벡터스토어 저장 완료.")

==> 4. 저장 → FAISS 벡터스토어에 저장
  FAISS 벡터스토어 저장 완료.


In [7]:
print("===> 5. 검색 → 질문과 유사한 문서 찾기")
# 성능을 위해 검색 결과 수 감소
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # 변경: 6 -> 4 (검색 결과 수 감소)
)
print("  검색기 설정 완료.")

===> 5. 검색 → 질문과 유사한 문서 찾기
  검색기 설정 완료.


In [8]:
print("===> 6. 생성 → LLM으로 답변 생성")
# LLM 모델 변경: ChatOpenAI -> ChatOllama
llm = ChatOllama(
    model="qwen3:1.7b",
    base_url="http://localhost:11434", # Ollama 서버 주소
    temperature=0.1,
    num_predict=1500  # 변경: max_tokens -> num_predict (Ollama에 맞는 파라미터)
)
print("  Ollama Qwen3 LLM 모델 로드 완료.")

===> 6. 생성 → LLM으로 답변 생성
  Ollama Qwen3 LLM 모델 로드 완료.


In [9]:
print("\n--- RAG 체인 설정 및 질문 시작 ---")

# 프롬프트 템플릿 정의
prompt_template = """
당신은 질문에 답변하는 친절한 챗봇입니다.
다음 문맥(Context)을 사용하여 질문에 답변해 주세요.

Context: {context}

Question: {question}

답변:"""
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# RetrievalQA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True, # 답변에 사용된 원본 문서 반환 설정
    chain_type_kwargs={"prompt": PROMPT}
)

# 예시 질문
question = "튜토리얼에 대한 설명을 해 줘"
result = qa_chain({"query": question})

print("\n--- 질문 ---")
print(question)
print("\n--- 답변 ---")
print(result["result"])
print("\n--- 출처 ---")
for doc in result["source_documents"]:
    print(f"- 페이지: {doc.metadata.get('page')}, 출처: {doc.metadata.get('source')}")

print("\n--- RAG 시스템 실행 완료 ---")


--- RAG 체인 설정 및 질문 시작 ---


C:\Users\user\AppData\Local\Temp\ipykernel_4336\1260895907.py:29: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})



--- 질문 ---
튜토리얼에 대한 설명을 해 줘

--- 답변 ---
<think>
Okay, let's see. The user is asking for an explanation of the tutorial. The context provided is a bit messy with different sections and numbers. Let me try to parse it.

First, the context starts with "7.4. 중지와 종료" which seems to be about stopping and ending. Then there's "8. 독립형 어플리케이션 생성" and "9. 애플릿 만들기" which are sections about creating standalone applications and apps. Then there's "3.4. 실행하기" and "3.5. 클래스 편집하기" etc., which are about running and editing classes. Finally, there's "12. 번역을 마치며" which is about translating and thanking God.

The user's question is about the tutorial, so I need to figure out what the tutorial is about based on the context. The main sections mention creating applications and apps, running them, editing classes, compiling, and translating. The tutorial seems to be a guide for developing applications using some programming tools, possibly Java or similar, given the terms like "애플릿" (applet) and "컴파일러" (com